In [ ]:
import pandas as pd

In [ ]:
npi = pd.read_csv('/ipsenhealthdata/npi/npidata_pfile_20050523-20191211.csv',
                  usecols=[
                      'NPI', 'Entity Type Code',
                      'Healthcare Provider Taxonomy Code_1', 'Provider Last Name (Legal Name)',
                      'Provider First Name', 'Provider Organization Name (Legal Business Name)',
                      'Provider First Line Business Practice Location Address',
                      'Provider Second Line Business Practice Location Address',
                      'Provider Business Practice Location Address City Name',
                      'Provider Business Practice Location Address State Name',
                      'Provider Business Practice Location Address Postal Code',
                      'Provider Business Practice Location Address Country Code (If outside U.S.)',
                      'Provider Business Practice Location Address Telephone Number'
                  ],
                  dtype={
                      'NPI':int,
                      'Entity Type Code':float,
                      'Healthcare Provider Taxonomy Code_1':str,
                      'Provider Last Name (Legal Name)':str,
                      'Provider First Name':str,
                      'Provider Organization Name (Legal Business Name)':str,
                      'Provider First Line Business Practice Location Address':str,
                      'Provider Second Line Business Practice Location Address':str,
                      'Provider Business Practice Location Address City Name':str,
                      'Provider Business Practice Location Address State Name':str,
                      'Provider Business Practice Location Address Postal Code':str,
                      'Provider Business Practice Location Address Telephone Number':str
                  },
                  sep=',')
npi = npi.rename(columns={
    'NPI':'npi',
    'Entity Type Code':'entity_type',
    'Healthcare Provider Taxonomy Code_1':'specialty_source_value',
    'Provider Last Name (Legal Name)':'last_name',
    'Provider First Name':'first_name',
    'Provider Organization Name (Legal Business Name)':'organization_name',
    'Provider First Line Business Practice Location Address':'address_1',
    'Provider Second Line Business Practice Location Address':'address_2',
    'Provider Business Practice Location Address City Name':'city',
    'Provider Business Practice Location Address State Name':'state',
    'Provider Business Practice Location Address Postal Code':'postal_code',
    'Provider Business Practice Location Address Country Code (If outside U.S.)':'country',
    'Provider Business Practice Location Address Telephone Number':'phone'
})
concept = pd.read_csv('s3://ipsenhealthdata/dev/anthony/physician_specialties/CONCEPT.csv',
                      sep='\t',
                      usecols=['concept_code', 'concept_id'])
npi = npi.merge(concept,
                how='left',
                left_on='specialty_source_value',
                right_on='concept_code')
del npi['specialty_source_value']
del npi['concept_code']
npi['concept_id'] = npi['concept_id'].fillna(0).astype(int)
concept_relationship = pd.read_csv('s3://ipsenhealthdata/dev/anthony/physician_specialties/CONCEPT_RELATIONSHIP.csv',
                      sep='\t', usecols=['concept_id_1', 'concept_id_2', 'relationship_id'])
concept_relationship = concept_relationship[concept_relationship['relationship_id']=='Maps to']
npi = npi.merge(concept_relationship[['concept_id_1', 'concept_id_2']],
                how='left', left_on='concept_id', right_on='concept_id_1')
del npi['concept_id']
del npi['concept_id_1']
npi['concept_id_2'] = npi['concept_id_2'].fillna(0).astype(int)
npi = npi.rename(columns={'concept_id_2':'specialty_concept_id'})
npi = npi.drop_duplicates()
npi['npi'] = npi['npi'].astype(float)